In [ ]:
from collections import defaultdict
import torch
import os   
import pandas as pd
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data, InMemoryDataset
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch.utils.data import random_split
from cap_dataset import CascadeRegression
from cap_model import GAT

In [ ]:
print(torch.__version__)

In [ ]:
print(os.getcwd())

In [ ]:
fb_regression_ds = CascadeRegression(root="data", name="facebook", edge_index_path="data/raw/facebook/adj.txt", task="regression", observation=3)

In [ ]:
model = GAT(fb_regression_ds.num_features, 64, 1, 8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [ ]:
total_size = len(fb_regression_ds)
train_size = int(0.7 * total_size)
valid_size = int(0.15 * total_size) 
test_size = total_size - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split (fb_regression_ds, [train_size, valid_size, test_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [ ]:
(train_loader.dataset[2].y.shape)

In [ ]:
def mrse_loss(output, target):
    # Check for zero to avoid division by zero
    if target != 0:
        loss = ((output - target) / target) ** 2
    else:
        # If target is zero, fallback to MSE to avoid NaN
        loss = (output - target) ** 2
    return loss

In [ ]:
def train(loader):
    total_loss = 0 
    model.train()
    print(len(loader))
    for idx, data in enumerate(loader):
        optimizer.zero_grad()
        out = model(data)
        print(data.cascade_name)
        print (out.shape)
        print(out)
        print(data.y.shape)
        print(data.y)
        print('\n')
        loss = F.mse_loss(out.squeeze().unsqueeze(0), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len (loader)

def test(loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            out = model(data)
            loss = F.mse_loss(out, data.y.view(-1, 1))
            total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
for epoch in range(100):
    train_loss = train(train_dataset)
    valid_loss = test(valid_dataset)
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')

test_loss = test(test_dataset)
print(f'Test MSE Loss: {test_loss:.4f}')

1. Reduce head to have better performance
2. Run 100 epochs 
3. Use graph norm 
4. See how coupledgnn data is 
5. see if there is an issue with the y and the output